In [2]:
# Configure the matplotlib graphics library and configure it to show
# figures inline in the notebook

%matplotlib inline
import matplotlib.pyplot as plt  # Import library for direct plotting functions
import numpy as np               # Import Numerical Python
from IPython.display import display, HTML #Import HTML for formatting output

# NOTE: Uncomment following lines ONLY if you are not using installation via pip
# import sys, os
# rootDir = '/path/to/arc/directory' # e.g. '/Users/Username/Desktop/ARC-Alkali-Rydberg-Calculator'
# sys.path.insert(0,rootDir)
# import sys, os
# sys.path.insert(0,"..")

import arc
from arc import *                 #Import ARC (Alkali Rydberg Calculator)

In [3]:
# -*- coding: utf-8 -*-
"""
Implements general single-atom calculations

This module calculates single (isolated) atom properties of all alkali metals in
general. For example, it calculates dipole matrix elements, quandrupole matrix
elements, etc.  Also, some helpful general functions are here, e.g. for saving
and loading calculations (single-atom and pair-state based), printing state
labels etc.


"""

from __future__ import division, print_function, absolute_import

from arc._database import sqlite3, UsedModulesARC
import csv
import gzip
from math import exp, sqrt
from mpmath import angerj

# for web-server execution, uncomment the following two lines
# import matplotlib
# matplotlib.use("Agg")
import numpy as np
import re
import shutil

from numpy.linalg import eigh

from scipy.constants import physical_constants, pi, epsilon_0, hbar
from scipy.constants import k as C_k
from scipy.constants import c as C_c
from scipy.constants import h as C_h
from scipy.constants import e as C_e
from scipy.constants import m_e as C_m_e

# for matrices
from numpy import floor

import sys
import os

if sys.version_info > (2,):
    xrange = range

import pickle


DPATH = os.path.join(os.path.expanduser("~"), ".arc-data")
__arc_data_version__ = 11

__all__ = [
    "AlkaliAtom",
    "printState",
    "printStateString",
    "printStateStringLatex",
    "printStateLetter",
    "formatNumberSI",
]


def setup_data_folder():
    """Setup the data folder in the users home directory."""
    if not os.path.exists(DPATH):
        os.makedirs(DPATH)

    # check what is the local version of data
    copyDataLocally = True
    versionFile = os.path.join(DPATH, "version.txt")
    if os.path.exists(versionFile):
        with open(versionFile, "r") as f:
            version = int(f.readline())
        if version == __arc_data_version__:
            copyDataLocally = False

    if copyDataLocally:
        dataFolder = os.path.join(
            os.path.dirname(os.path.realpath(__file__)), "data"
        )
        for fn in os.listdir(dataFolder):
            if os.path.isfile(os.path.join(dataFolder, fn)):
                shutil.copy(os.path.join(dataFolder, fn), DPATH)

        dataFolder = os.path.join(dataFolder, "refractive_index_data")
        refractiveIndexData = os.path.join(DPATH, "refractive_index_data")

        if not os.path.exists(refractiveIndexData):
            os.makedirs(refractiveIndexData)

        for fn in os.listdir(dataFolder):
            if os.path.isfile(os.path.join(dataFolder, fn)):
                shutil.copy(os.path.join(dataFolder, fn), refractiveIndexData)

        with open(versionFile, "w") as f:
            f.write("%d" % __arc_data_version__)

In [4]:
def getEnergy(self, n, l, j, s=0.5):
        """
        Energy of the level relative to the ionisation level (in eV)

        Returned energies are with respect to the center of gravity of the
        hyperfine-split states.
        If `preferQuantumDefects` =False (set during initialization)
        program will try use NIST energy value, if such exists,
        falling back to energy calculation with quantum defects if
        the measured value doesn't exist. For `preferQuantumDefects` =True,
        program will calculate energies from quantum defects
        (useful for comparing quantum defect calculations with measured
        energy level values) if the principal quantum number of the
        requested state is larger than the minimal quantum principal quantum
        number `self.minQuantumDefectN` which sets minimal quantum number
        for which quantum defects still give good estimate of state energy
        (below this value saved energies will be used if existing).

        Args:
            n (int): principal quantum number
            l (int): orbital angular momentum
            j (float): total angular momentum
            s (float): optional, total spin angular momentum. Default value
                of 0.5 is correct for Alkali atoms, and has to be specified
                explicitly for divalent atoms.

        Returns:
            float: state energy (eV)
        """
        if l >= n:
            raise ValueError(
                "Requested energy for state l=%d >= n=%d !" % (l, n)
            )

        # use NIST data ?
        if (
            (not self.preferQuantumDefects or n < self.minQuantumDefectN)
            and (n <= self.NISTdataLevels)
            and (abs(self._getSavedEnergy(n, l, j, s=s)) > 1e-8)
        ):
            return self._getSavedEnergy(n, l, j, s=s)

        # else, use quantum defects
        defect = self.getQuantumDefect(n, l, j, s=s)
        return -self.scaledRydbergConstant / ((n - defect) ** 2)


def _getSavedEnergy(self, n, l, j, s=0.5):
        if abs(j - (l - 0.5)) < 0.001:
            # j = l-1/2
            return self.sEnergy[n, l]
        elif abs(j - (l + 0.5)) < 0.001:
            # j =l+1/2
            return self.sEnergy[l, n]
        else:
            raise ValueError(
                "j (=%.1f) is not equal to l+1/2 nor l-1/2 (l=%d)" % (j, l)
            )


def getQuantumDefect(self, n, l, j, s=0.5):
        """
        Quantum defect of the level.

        For an example, see `Rydberg energy levels example snippet`_.

        .. _`Rydberg energy levels example snippet`:
            ./Rydberg_atoms_a_primer.html#Rydberg-Atom-Energy-Levels

        Args:
            n (int): principal quantum number
            l (int): orbital angular momentum
            j (float): total angular momentum
            s (float): (optional). Total spin angular momentum.
                Default value of 0.5 correct for Alkali atoms. For divalent
                atoms it has to be explicitly defined.

        Returns:
            float: quantum defect
        """
        defect = 0.0
        if l < 5:
            # find correct part in table of quantum defects
            modifiedRRcoef = self.quantumDefect[round(floor(s) + s + j - l)][l]
            if l < 3 and abs(modifiedRRcoef[0]) < 1e-9 and self.Z != 1:
                # it's not Hydrogen but for l in {s,p,d} quantum defect is 0
                raise ValueError(
                    "Quantum defects for requested state "
                    + ("(n = %d, l = %d, j = %.1f, s=%.1f) are" % (n, l, j, s))
                    + " uknown. Aborting calculation."
                )
            defect = (
                modifiedRRcoef[0]
                + modifiedRRcoef[1] / ((n - modifiedRRcoef[0]) ** 2)
                + modifiedRRcoef[2] / ((n - modifiedRRcoef[0]) ** 4)
                + modifiedRRcoef[3] / ((n - modifiedRRcoef[0]) ** 6)
                + modifiedRRcoef[4] / ((n - modifiedRRcoef[0]) ** 8)
                + modifiedRRcoef[5] / ((n - modifiedRRcoef[0]) ** 10)
            )
        else:
            # use \delta_\ell = \delta_g * (4/\ell)**5
            # from https://journals.aps.org/pra/abstract/10.1103/PhysRevA.74.062712
            defect = self.quantumDefect[0][4][0] * (4 / l) ** 5
        return defect

In [5]:
print(getEnergy(1, 5, 0, 1/2, 1/2))

AttributeError: 'int' object has no attribute 'preferQuantumDefects'

In [6]:
calculator = RydbergCalculator()
energy = calculator.getEnergy(1, 5, 0, 0.5, 0.5)
print(energy)


NameError: name 'RydbergCalculator' is not defined

In [7]:
from arc import *

atom = Rubidium()


In [39]:
# Rb 원자의 에너지 레벨 계산
energy1 = atom.getEnergy(n=5, l=0, j=1/2,s=1/2)
energy2 = atom.getEnergy(n=5, l=1, j=1/2,s=1/2)
print(f"The energy of the 5S1/2 state is {energy1} eV")
print(f"The energy of the 5P1/2 state is {energy2} eV")


RydbergConstant = atom.scaledRydbergConstant
energy5S = -RydbergConstant / ((5- defect1) ** 2)
print(energy5S)

The energy of the 5S1/2 state is -4.177126523766669 eV
The energy of the 5P1/2 state is -2.6175355237666684 eV
-4.6177032511225065


In [9]:
C_h*C_c / (abs(energy1-energy2)*C_e)

7.949789299450962e-07

In [10]:
abs(energy1-energy2)

1.5595910000000002

In [35]:
defect1 = atom.getQuantumDefect(n=5, l=0, j=1/2, s=1/2)
defect2 = atom.getQuantumDefect(n=5, l=1, j=1/2)

print(defect1)
print(defect2)

defect1_2 = atom.getQuantumDefect(n=5, l=2, j=3/2)
defect2_2 = atom.getQuantumDefect(n=5, l=3, j=1/2)
print(defect1_2)
print(defect2_2)

defect_total_0 = defect1 + (defect1_2 / ((5-defect1)**2))
defect_total_1 = defect2 + (defect2_2 / (5-defect2)**2)

print(defect_total_0)
print(defect_total_1)

3.1822614821801163
2.7076163449135167
1.302887722724843
0.013096622058855924
3.270029275206541
2.710108555223957


In [21]:
print(C_h)
print(C_e)
print(C_c)

6.62607015e-34
1.602176634e-19
299792458.0


In [22]:
print(f"Scaled Rydberg constant for Rubidium,: {atom.scaledRydbergConstant} eV")


Scaled Rydberg constant for Rubidium,: 13.605605218602195 eV


In [18]:
3.13109 + (0.204 / (5-3.13109)**2) + (-1.8 / (5-313109)**4)

3.1894954523364913